# CS295B F18: Homework 7
## Differentially Private Machine Learning

## Instructions

Before you start, download the example dataset and ensure that all cells in this notebook execute without error. If you have trouble getting the notebook to run, please post a question on Piazza.

To ensure that the notebook runs, I've defined a function `your_code_here()` that simply returns the number `1`. Whenever you see a call to this function, you should replace it with code you have written. Please make sure all cells of your notebook run without error before submitting the assignment. If you have not completed all the questions, leave calls to `your_code_here()` in place or insert dummy values so that the cell does not throw an error when it runs.

To help you arrive at the correct solution, I have left the value computed by my solution in the uploaded version of this notebook. You can refer to these example results by viewing the notebook on Github. If you re-run the cell after downloading the notebook, the results will disappear (because the notebook no longer contains the code that generated them). Your solutions should produce results similar to the ones in the uploaded notebook.

When answering non-code questions, feel free to use a comment, or put the cell in Markdown mode and use Markdown.

The assignment is due by 11:59pm on Monday, November 12. When you have finished your assignment, submit it via Blackboard under the assignment "Homework 6." For questions on grading and submitting assignments, refer to the course webpage or email the instructor.

The dataset files you'll need are available here:

- [`adult_processed_x`](https://github.com/jnear/cs295-data-privacy/blob/master/slides/adult_processed_x.npy)
- [`adult_processed_y`](https://github.com/jnear/cs295-data-privacy/blob/master/slides/adult_processed_y.npy)

## Collaboration Statement

In the cell below, write your collaboration statement. This statement should describe all collaborations, even high-level ones (e.g. "I discussed my general approach for answering question 3 with Josh"). High-level collaborations of this kind are allowed as long as they are described; copying of answers or code is not allowed.

In [1]:
# Write your collaboration statement here

--------------------

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import pandas as pd
import numpy as np

# Some useful utilities

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

def z_clip(xs, b):
    return [min(x, b) for x in xs]

def clip(xs, upper, lower):
    return [max(min(x, upper), lower) for x in xs]

def gaussian_mech_vec(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon, size=len(v))

def g_clip(v):
    n = np.linalg.norm(v, ord=2)
    if n > 1:
        return v / n
    else:
        return v

def your_code_here():
    return 1

def test(msg, value, expected):
    if value == expected:
        print(f"{msg}: {value}, as expected")
    else:
        print(f"{msg}: OH NO! Got {value}, but expected {expected}.")

In [3]:
X = np.load('adult_processed_x.npy')
y = np.load('adult_processed_y.npy')

training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

In [4]:
# Prediction: take a model (theta) and a single example (xi) and return its predicted label
def predict(theta, xi):
    label = np.sign(xi @ theta)
    return label

# The loss function measures how good our model is. The training goal is to minimize the loss.
# This is the average logistic loss function.
def loss(theta, x, y, lambda_param=0):
    exponent = - y * (x.dot(theta))
    regularization = (lambda_param/2) * np.sum(theta*theta)
    return np.sum(np.log(1+np.exp(exponent))) / x.shape[0]

# This is the average gradient of the logistic loss
# The gradient is a vector that indicates in which direction the loss function is increasing fastest
def gradient(theta, x, y, lambda_param=0):
    exponent = y * (x.dot(theta))
    gradient_loss = - (np.transpose(x) @ (y / (1+np.exp(exponent)))) / (x.shape[0])
    regularization = lambda_param * theta
    return gradient_loss + regularization

# our measure of accuracy is just % correct of the test set
def accuracy(theta, X, y):
    return np.sum([predict(theta, xi) for xi in X] == y) / y.shape[0]

# This is gradient descent with a *learning rate* "eta"
def gradient_descent(X_train, y_train, iterations, status = False):
    theta = np.zeros(X_train.shape[1])
    eta = 1.0
    lambda_param = 0.001

    for i in range(iterations):
        theta = theta - eta * gradient(theta, X_train, y_train, lambda_param)
        if status and i % int(iterations / 5) == 0:
            print(f"Iteration {i}: loss = {loss(theta, X_train, y_train, lambda_param)}")

    return theta

In [5]:
theta = gradient_descent(X_train, y_train, 1000, True)
acc = accuracy(theta, X_test, y_test)
print(f"Final accuracy: {acc}")

Iteration 0: loss = 0.549109439168421
Iteration 200: loss = 0.37016593673141646
Iteration 400: loss = 0.36183446200459896
Iteration 600: loss = 0.3580948174586848
Iteration 800: loss = 0.3560122225542519
Final accuracy: 0.8316010614772225


----------------

## Question 1 (10 points)

Implement a function `dp_gradient_descent` that performs differentially private gradient descent by adding noise to the gradient at each iteration. Your function should take additional arguments $\epsilon$ and $\delta$, and should have an **overall** privacy cost of $(\epsilon, \delta)$-differential privacy.

*Hint*: Use `gaussian_mech_vec`, defined above, to add noise.

*Hint*: Use advanced composition to bound the total privacy cost. Start with the total privacy cost of $k$-fold adaptive composition under advanced composition, then solve for $\epsilon_i$, the privacy cost per iteration. Use this result to set the per-iteration value of `epsilon`, and similar for `delta`.

*Hint*: Don't forget clipping!

In [10]:
def dp_gradient_descent(X_train, y_train, iterations, epsilon, delta):
    return your_code_here()

delta = 1e-5
X_test_clip = np.array([g_clip(x) for x in X_test])

for epsilon in [0.01, 0.1, 1.0]:
    theta = dp_gradient_descent(X_train, y_train, 500, epsilon, delta)
    acc = accuracy(theta, X_test_clip, y_test)
    print(f"Epsilon = {epsilon}, final accuracy: {acc}")

Epsilon = 0.01, final accuracy: 0.6167624944714728
Epsilon = 0.1, final accuracy: 0.731203007518797
Epsilon = 1.0, final accuracy: 0.8032950022114109


## Question 2 (5 points)

In 2-5 sentences, argue that your implementation of `dp_gradient_descent` satisfies $(\epsilon, \delta)$-differential privacy.

## Question 3 (10 points)

Implement a function `zcdp_gradient_descent` that performs differentially private gradient descent by adding noise to the gradient at each iteration. Your function should take an additional argument $\rho$, and should have an **overall** privacy cost of $\rho$-zero concentrated differential privacy. You will also have to implement `gaussian_mech_vec_zcdp`, the vector-valued gaussian mechanism for zCDP.

In [7]:
def gaussian_mech_vec_zcdp(v, sensitivity, rho):
    return your_code_here()

def zcdp_gradient_descent(X_train, y_train, iterations, rho):
    return your_code_here()

X_test_clip = np.array([g_clip(x) for x in X_test])

for rho in [0.00001, 0.0001, 0.001]:
    theta = zcdp_gradient_descent(X_train, y_train, 500, rho)
    acc = accuracy(theta, X_test_clip, y_test)
    print(f"rho = {rho}, final accuracy: {acc}")

rho = 1e-05, final accuracy: 0.7622733303847855
rho = 0.0001, final accuracy: 0.8143520566121185
rho = 0.001, final accuracy: 0.8198805838124723


## Question 4 (5 points)

Implement a function `convert_zCDP_eps_delta` that converts a $\rho$-zCDP privacy bound to a $(\epsilon,\delta)$-differential privacy bound, given a target $\delta$.

In [8]:
def convert_zCDP_eps_delta(rho, delta):
    return your_code_here()

delta = 1e-5

for rho in [0.00001, 0.0001, 0.001]:
    theta = zcdp_gradient_descent(X_train, y_train, 500, rho)
    acc = accuracy(theta, X_test_clip, y_test)
    epsilon = convert_zCDP_eps_delta(rho, delta)
    print(f"rho = {rho}, epsilon = {epsilon}, final accuracy: {acc}")

rho = 1e-05, epsilon = 0.021469660262893472, final accuracy: 0.7585139318885449
rho = 0.0001, epsilon = 0.06796140424415112, final accuracy: 0.7999778858911986
rho = 0.001, epsilon = 0.21559660262893474, final accuracy: 0.8155683325961963


## Question 5 (10 points)

Implement a function `output_perturbation` that performs differentially private machine learning by adding noise to the **final model** (i.e. **output perturbation**). Your function should take additional arguments $\epsilon$ and $\delta$, and should have an **overall** privacy cost of $(\epsilon,\delta)$-differential privacy.

*Hint*: use `gradient_descent` to do the machine learning, then `gaussian_mech_vec` to add noise to the `theta` which comes out. Recall the sensitivity of ERM from class.

**Note**: as mentioned in class, this approach does *not* actually satisfy differential privacy unless `gradient_descent` reaches the true minimizer. For this problem, assume that it does.

In [9]:
def output_perturbation(X_train, y_train, iterations, epsilon, delta):
    return your_code_here()

delta = 1e-5
X_test_clip = np.array([g_clip(x) for x in X_test])

for epsilon in [0.01, 0.1, 1.0]:
    theta = output_perturbation(X_train, y_train, 500, epsilon, delta)
    acc = accuracy(theta, X_test_clip, y_test)
    print(f"Epsilon = {epsilon}, final accuracy: {acc}")

Epsilon = 0.01, final accuracy: 0.49159663865546216
Epsilon = 0.1, final accuracy: 0.7533171163202123
Epsilon = 1.0, final accuracy: 0.8176691729323309


## Question 6 (5 points)

How could `gradient_descent` be re-defined so that `output_perturbation` actually leaks information about a single sample in the training data? (i.e. how could `output_perturbation` violate privacy in a severe way)

## Question 7 (5 points)
Which of the approaches you have implemented is likely to yield the best accuracy in practice? (ignore the non-privacy of `output_perturbation` for this question)